# Generate flow
The core of simulation is the reproduction and intervention of traffic demand, that is, the data and related information of traffic flow. In SUMO, traffic demands can be defined as individual vehicle flows and routes. Basic information should include departure time, departure place, destination and mode of access, which can be different types of vehicles or pedestrians. When traffic needs are defined as routes, information about origin and destination is not mandatory. 

Once the traffic demand is generated, the traffic distribution procedures in SUMO can be executed, including DUAROUTER, MAROUTER and Oneshot. The above allocation methods are different under different allocation principles such as User Equalization (UE), Random User Equalization (SUE) or the fastest path for a given departure time. The final traffic requirements are stored in the form of xml files. Similarly, we can also directly modify the xml file to set the traffic requirements we need. The interaction between the vehicles is set by the car-following model and the lane-changing model. The Krauss model is used by default, and researchers can also use IDM, IDMM, ACC, CACC and other car-following models, or LC2013, SL2015 and other lane-changing models according to their needs.

## Get lane/edge ID of target flow
1. Run <SUMO_HOME>/bin/netedit.exe
2. Click File -> Open Network and select the corresponding osm.net.xml file
3. Select Inspect (as shown below), click the target lane, and you can get its ID on the left panel
4. Save and exit

- Note 1: Rename the ID of the target road to a form that is easy to remember, such as: "in + number", in1
- Note 2: Run build.bat to ensure that the generated trip file contains the correct edge ID

## Traffic flow injection

Configure the target road to inject traffic and the number of vehicles

In [125]:
##################################
# configuration parameters
##################################
# Simulation file path
folder = '../bigger demo# with sensor v2/'

# Set the target road to inject traffic flow
newTripsFrom = ['in1', 'in2', 'in3', 'in4']

# Set the amount of traffic injected during the simulation
newTripsNum = [0, 0, 0, 1000]

Run the script to add traffic flow

In [126]:
##################################
# Count existing trips
##################################
from xml.dom import minidom
DOMTree = minidom.parse(folder + 'osm.passenger.trips.xml')
collection = DOMTree.documentElement

# Count existing trips
mergedTrips = collection.getElementsByTagName('trip')

# get all destinations
allDest = {}
for trip in mergedTrips:
     dest = trip.getAttribute('to')
     if dest not in allDest:
         allDest[dest] = True
allDest = list(allDest)

In [127]:
##################################
# generate new trips
##################################
import random
lastTrip = mergedTrips[-1]
newId = int(lastTrip.getAttribute('id')[3:]) + 1
endAt = float(lastTrip.getAttribute('depart'))

for n in range(len(newTripsFrom)):
    edge = newTripsFrom[n]
    newTripNum = newTripsNum[n]
    if newTripNum <= 0:
        newTripNum = 1
    for i in range(newTripNum):
        newTrip = lastTrip.cloneNode(True)
        newTrip.setAttribute('id', 'veh%d' % newId)
        newTrip.setAttribute('depart', '%.2f' % ((endAt / newTripNum) * i)) # set time interval
        newTrip.setAttribute('from', edge)
        newTrip.setAttribute('to', random.choice(allDest))
        newId += 1
        mergedTrips.append(newTrip)
        
# After the fused trip, its part time must be arranged in ascending order
mergedTrips = sorted(mergedTrips, key=lambda x:float(x.getAttribute('depart')), reverse=False) 

In [128]:
##################################
# output to file
##################################
mergedTripCollection = collection.cloneNode(False)
vType = collection.getElementsByTagName('vType')[0]
mergedTripCollection.appendChild(vType)

for trip in mergedTrips:
    mergedTripCollection.appendChild(trip)

# replace all previous trips
for i in range(len(DOMTree.childNodes)):
    node = DOMTree.childNodes[i]
    if isinstance(node, minidom.Element):
        DOMTree.childNodes[i] = mergedTripCollection

# to file
try:
    with open(folder + 'test.trips.xml','w',encoding='UTF-8') as fh:
        # The first parameter of writexml is the target file object, the second parameter is the indentation format of the root node, 
        # and the third parameter is the indentation format of other child nodes. 
        # The fourth parameter specifies the newline format, and the fifth parameter specifies the encoding of the xml content.
        DOMTree.writexml(fh, indent='', addindent='\t',newl='\n',encoding='UTF-8')
        print('OK')
except Exception as err:
    print('Error：{err}'.format(err=err))


OK
